In [1]:
import sys
import artm

In [37]:
import re

with open('../data/interim/articles_vw.txt') as fl:
    data = fl.read().replace('2:11.кать', '')
with open('../data/interim/articles_vw.txt', 'w') as fl:
    fl.write(data)

In [38]:
batch_vectorizer = artm.BatchVectorizer(data_path='../data/interim/articles_vw.txt',
                                        data_format='vowpal_wabbit',
                                        target_folder='../data/interim/batches')

InvalidOperationException: Error in ../data/interim/articles_vw.txt:6192, can not parse integer number of occurences: 1:0.болельщик

In [20]:
'ответил:-'
'фото:/ru'
'2:1.самый'
'1:11.самый'
'18:00.читать'
'применяются):маршрут:3'
'4:2.через'

'применяются):маршрут:3'

In [ ]:


dictionary = artm.Dictionary()
dictionary.gather(data_path='my_collection_batches')